# PageRank

This page demonstrates the use of a short Python implementation of the PageRank algorithm on the link structure contained in the graph on the [PageRank Wikipedia](http://en.wikipedia.org/wiki/PageRank) page:

In [2]:
from IPython.display import Image
Image(url='http://upload.wikimedia.org/wikipedia/commons/f/fb/PageRanks-Example.svg')

In [4]:
import numpy as np

First, we will encode the links present on this graph as a count matrix `M_counts`.

In [5]:
n_pages = 11 # numbering pages A through K as 0 to 10
M_counts = np.zeros((n_pages, n_pages)) # will hold the number of link counts (assumed 0 or 1)
# columns = starting page, row = destination page, ie M_ij = whether or not there is a link from j to i

M_counts[:,0] = 1 # page 0 (A in the graphic) is a sink because it has no outgoing links at all; 
# however, M cannot contain an all-zero column, so do as if A was linking to all other pages (ie put 1's everywhere)
M_counts[2,1] = 1 # B->C
M_counts[1,2] = 1 # C->B
M_counts[0,3] = 1 # D->A
M_counts[1,3] = 1 # D->B
M_counts[1,4] = 1 # E->B
M_counts[3,4] = 1 # E->D
M_counts[5,4] = 1 # E->F
M_counts[1,5] = 1 # F->B
M_counts[4,5] = 1 # F->E
M_counts[1,6] = 1 # G,H,I->B,E
M_counts[4,6] = 1
M_counts[1,7] = 1
M_counts[4,7] = 1
M_counts[1,8] = 1
M_counts[4,8] = 1
M_counts[4,9] = 1 # J,K->E
M_counts[4,10] = 1
print(M_counts)

[[ 1.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.]
 [ 1.  0.  1.  1.  1.  1.  1.  1.  1.  0.  0.]
 [ 1.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 1.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.]
 [ 1.  0.  0.  0.  0.  1.  1.  1.  1.  1.  1.]
 [ 1.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.]
 [ 1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]]


Now we can make an adjacency matrix `M` out of `M_counts`, by dividing each column by its sum, ie we are making sure columns sum to 1 :

In [6]:
M = np.empty((n_pages, n_pages))
for j in range(n_pages):
    M[:,j] = M_counts[:,j] / M_counts[:,j].sum()
np.set_printoptions(precision=3)
print(M)

[[ 0.091  0.     0.     0.5    0.     0.     0.     0.     0.     0.     0.   ]
 [ 0.091  0.     1.     0.5    0.333  0.5    0.5    0.5    0.5    0.     0.   ]
 [ 0.091  1.     0.     0.     0.     0.     0.     0.     0.     0.     0.   ]
 [ 0.091  0.     0.     0.     0.333  0.     0.     0.     0.     0.     0.   ]
 [ 0.091  0.     0.     0.     0.     0.5    0.5    0.5    0.5    1.     1.   ]
 [ 0.091  0.     0.     0.     0.333  0.     0.     0.     0.     0.     0.   ]
 [ 0.091  0.     0.     0.     0.     0.     0.     0.     0.     0.     0.   ]
 [ 0.091  0.     0.     0.     0.     0.     0.     0.     0.     0.     0.   ]
 [ 0.091  0.     0.     0.     0.     0.     0.     0.     0.     0.     0.   ]
 [ 0.091  0.     0.     0.     0.     0.     0.     0.     0.     0.     0.   ]
 [ 0.091  0.     0.     0.     0.     0.     0.     0.     0.     0.     0.   ]]


Let us check that all the conditions on M are fulfilled.

In [7]:
import numpy
def check_M(M):
    """
    check that M has the right format to be used by pagerank function
    """
    n_pages = M.shape[0] # n_pages is the number of rows of M
    np.testing.assert_equal(M.shape[0], M.shape[1], err_msg = 'M should be square')
    np.testing.assert_array_almost_equal(M.sum(axis=0), np.ones((n_pages)), 
                                         err_msg = 'assert each column sums to one (M is assumed column-stochastic)')
    for j in range(n_pages):
        M_column = M[:,j]
        n_nonzero = np.count_nonzero(M[:,j])
        np.testing.assert_array_almost_equal(M_column[M_column.nonzero()], np.ones((n_nonzero)) / n_nonzero,
                                             err_msg = 'in column %g, all non-zero entries should be equal (and equal to 1 divided by their number)' % j)

check_M(M) # will produce error if M does not have the right format

And we are now ready to apply the `pagerank` function, which will iteratively apply page transitions to an randomly initialized distribution over the pages, until convergence.

In [8]:
import numpy as np
def pagerank(M, d=0.85, square_error=1e-6):
    """
    M : the adjacency matrix of the pages. It is assumed to be column-stochastic (ie column sum to 1); all links have equal weight.
    A page with no outgoing links (sink) is represented as a page with outgoing links to each other page (ie restart page).
    d: damping factor
    square_error : the algorithm iterates until the difference between two successive PageRank vectors v is less than this (in squared norm)
    returns the PageRanks of all pages
    """
    n_pages = M.shape[0] # n_pages is the number of rows of M
    v = np.random.rand(n_pages) # initialize to random vector
    v = v / v.sum() # make v sum to 1
    last_v = np.ones((n_pages)) # will contain the previous v
    M_hat = d * M + (1-d)/n_pages * np.ones((n_pages, n_pages)) # equation (***) in Wikipedia page
    while np.square(v - last_v).sum() > square_error:
        last_v = v
        v = M_hat.dot(v) # at each iteration, progress one timestep
    return v
    

In [9]:
pagerank(M)

array([ 0.033,  0.384,  0.343,  0.039,  0.081,  0.039,  0.016,  0.016,
        0.016,  0.016,  0.016])

These are the numbers (within the allowed error) displayed on the graph (the numbers on the graph are rounded exact values).